<a href="https://colab.research.google.com/github/Jessicawwww/Capstone-Experiments/blob/main/Capstone_Experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Mounting Google Drive onto virtual environment
from google.colab import drive
drive.mount('/content/drive')
# #Installing Base Packages
# !pip3 install utils
# !pip3 install numpy==1.17.0
# !pip3 install scipy==1.4.1

Mounted at /content/drive


In [ ]:
#/content/drive/My Drive/ gets to your drive. 
%cd /content/drive/My Drive/Capstone TCS/final

/content/drive/.shortcut-targets-by-id/1DzBEX3L6TIcOWVCpjtc2tmWd8buAK-zO/Capstone TCS/final


In [ ]:
# %ls

## EDA (to be continued)

In [ ]:
!pip install tensorflow_text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 45.0 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import tempfile
import os

In [ ]:
import pandas as pd
import glob

schema = {
    'song_hotttnesss': float,
    'artist_familiarity': float,
    'artist_hotttnesss': float,
    'artist_id': str,
    'artist_latitude': float,
    'artist_location': str,
    'artist_longitude': float,
    'artist_name': str,
    'title': str,
    'danceability': float,
    'duration': float,
    'end_of_fade_in': float,
    'energy': float,
    'key': float,
    'key_confidence': float,
    'loudness': float,
    'mode': float,
    'mode_confidence': float,
    'start_of_fade_out': float,
    'tempo': float,
    'time_signature': float,
    'time_signature_confidence': float,
    'artist_terms': str,
    'artist_terms_freq': str,
    'artist_terms_weight': str,
    'year': float
}

# read each CSV file into a separate DataFrame, and concatenate them
cols=list(schema.keys())
temp=[]
# for csv_file in glob.glob('/Users/olivia/Downloads/final/*.csv'):
for csv_file in glob.glob('*.csv'):
    temp_df = pd.read_csv(csv_file,names=cols)
    temp.append(temp_df)
    
df = pd.concat(temp, ignore_index=True)

In [ ]:
len(df)

In [ ]:
df.columns

In [ ]:
df.danceability.value_counts()

In [ ]:
df.energy.value_counts()

In [ ]:
# drop empty features
df.drop(['danceability','energy'],axis=1,inplace=True)

In [ ]:
df.head()

In [ ]:
for col in df.columns:
    num_nans = df[col].isnull().sum()
    print(f"Number of NaN values in column '{col}': {num_nans}")

We define a song to be popular if its song_hotttnesss value is above average,and assign a label called popularity

In [ ]:
import numpy as np
pop_avg=np.mean(df['song_hotttnesss'])
df['popularity']= df.song_hotttnesss.map(lambda x: 1 if x > pop_avg else 0)

**Here is one way to split the data if you want a pre/post deploy differentiation**

In [ ]:
# define the three columns to split by
split_cols = ['artist_latitude', 'artist_longitude']


# Split the dataframe into two parts based on the null values in split_cols
df_notnull = df[df[split_cols].notnull().all(axis=1)]
df_null = df[~df[split_cols].notnull().all(axis=1)]

In [ ]:
len(df_null)

In [ ]:
len(df_notnull)

In [ ]:
df_notnull.columns

In [ ]:
labels = ['artist_familiarity', 'artist_hotttnesss', 'duration', 'end_of_fade_in', 'key', 
            'key_confidence', 'loudness', 'mode', 'mode_confidence', 'start_of_fade_out', 'tempo', 
            'time_signature', 'time_signature_confidence','year', 'artist_latitude', 'artist_longitude','popularity']

In [ ]:
#Drop 'artist_familiarity' NA
# pre_data = df_notnull[labels]
# post_data = df_null[labels]
# post_data[['artist_latitude', 'artist_longitude']] = post_data[['artist_latitude', 'artist_longitude']].fillna(0)
# pre_data = pre_data.dropna()
# post_data = post_data.dropna()

post_data = df_notnull[labels]
pre_data = df_null[labels]
pre_data[['artist_latitude', 'artist_longitude']] = pre_data[['artist_latitude', 'artist_longitude']].fillna(0)
pre_data = pre_data.dropna()
post_data = post_data.dropna()

In [ ]:
len(labels)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
for i in range(0,16):
    plt.figure(figsize=(3,2))
    plt.hist(pre_data[labels[i]], bins = 100, alpha = 0.5)
    plt.hist(post_data[labels[i]], bins = 100, alpha = 0.5)
    plt.xlabel('Values')
    plt.ylabel('Frequency')
    plt.title('Column '+ str(i+1))
    plt.legend(['prev', 'latter'])
    plt.show()

In [ ]:
#Label distribution
plt.figure(figsize=(3,2))
plt.hist(pre_data['popularity'], bins = 100, alpha = 0.5)
plt.hist(post_data['popularity'], bins = 100, alpha = 0.5)
plt.xlabel('Values')
plt.ylabel('Frequency')
plt.title('Column label')
plt.legend(['prev', 'latter'])
plt.show()

In [ ]:
# !pip3 install -U scikit-learn scipy matplotlib

In [ ]:
# !pip install --upgrade pip

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import neighbors
from sklearn.metrics import mean_squared_error 
from math import sqrt
import joblib

## Training demo

We define a song to be popular if its song_hotttnesss value is above average,

In [ ]:
numeric_features= ['artist_familiarity', 'artist_hotttnesss', 'duration', 'end_of_fade_in', 'key', 
            'key_confidence', 'loudness', 'mode', 'mode_confidence', 'start_of_fade_out', 'tempo', 
            'time_signature', 'time_signature_confidence','year', 'artist_latitude', 'artist_longitude']
label= ['popularity']

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix

# Split data into training and testing sets
df_sample=pre_data[numeric_features+label].dropna()
X = df_sample[numeric_features]
y = df_sample[label]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# scaler = MinMaxScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

In [ ]:
# Instantiate a Random Forest Classifier with 100 estimators
rf = RandomForestClassifier(random_state=42)

# Fit the model to the training data
rf.fit(X_train, y_train)

# Predict the labels of the test data
y_pred = rf.predict(X_test)

# Compute accuracy score
accuracy = accuracy_score(y_test, y_pred)
print("Random Forest Classifier Accuracy:", accuracy)

In [ ]:
# Calculate the confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix")
print(cm)
print("Model benchmark")
benchmark = accuracy
print(benchmark)

In [ ]:
Model_directory = tempfile.gettempdir()

In [ ]:
Model_directory

In [ ]:
version = 1

In [ ]:
export_path = os.path.join(Model_directory,str(version))
print('export_path = {}\n'.format(export_path))

In [ ]:
#save model
import joblib
joblib.dump(rf, "./deployed_model.joblib")
rf.save(export_path)

In [ ]:
import joblib
deployed_model = joblib.load("./deployed_model.joblib")

In [ ]:
import pickle
pickle.dump(deployed_model, open("rfc.pkl", 'wb'))

In [ ]:
retrained_model = joblib.load("./retrained_model.joblib")
pickle.dump(retrained_model, open("retrained_model.pkl", 'wb'))

/usr/local/lib/python3.9/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.2.0 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.2.0 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomizedSearchCV from version 1.2.0 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your

In [ ]:
deployed_model

In [ ]:
print(len(pre_data))
print(len(post_data))

In [ ]:
# Predict the labels of the test data
y_pred_tr = deployed_model.predict(X_train)
y_pred_test = deployed_model.predict(X_test)

# Compute accuracy score
train_score = accuracy_score(y_train, y_pred_tr) * 100
print(f"Train accuracy score: {train_score:.2f}%")
test_score = accuracy_score(y_test, y_pred_test) * 100
print(f"Test accuracy score: {test_score:.2f}%")

In [ ]:
np.unique(y_test)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

cm = confusion_matrix(y_test, y_pred_test)
disp = ConfusionMatrixDisplay(
    confusion_matrix=cm,
    display_labels=[0,1]
)


# NOTE: Fill all variables here with default values of the plot_confusion_matrix
fig, ax = plt.subplots(figsize=(10, 10))
disp = disp.plot(xticks_rotation='vertical', ax=ax, cmap="summer")

plt.show()

In [ ]:
pd.DataFrame(classification_report(y_test, y_pred_test, output_dict=True)).T

## Change of Data Schema

In [ ]:
def test_data_schema(pre_data, post_data):
    assert set(pre_data.columns) == set(post_data.columns),"Data Schema shifted."

In [ ]:
test_data_schema(pre_data, post_data)

This indicates that there is no data schema change for our dataset.

## Amount of new data

In [ ]:
new_data_func = lambda pre, post: True if len(post)>len(pre) else False

In [ ]:
new_data_func(pre_data, post_data)

This indicates that post data has more rows than pre data. This is expected as need to retrain the model with post data added.

## Concept Drift: shift in relationship between X and y

In [ ]:
print(len(post_data), len(pre_data))

In [ ]:
X_post = post_data[numeric_features]
y_post = post_data[label]
y_pred_post = deployed_model.predict(X_post)
# X_post = scaler.transform(X_post)
# Compute accuracy score
post_score = accuracy_score(y_post, y_pred_post) * 100
print(f"Post data accuracy score: {post_score:.2f}%")

In [ ]:
print(len(X_train), len(X_test), len(X_post))

In [ ]:
cm_post = confusion_matrix(y_post, y_pred_post)
disp_post = ConfusionMatrixDisplay(
    confusion_matrix=cm_post,
    display_labels=[0,1]
)


# NOTE: Fill all variables here with default values of the plot_confusion_matrix
fig, ax = plt.subplots(figsize=(8, 8))
disp_post = disp_post.plot(xticks_rotation='vertical', ax=ax, cmap="summer")

plt.show()

In [ ]:
pd.DataFrame(classification_report(y_post, y_pred_post, output_dict=True)).T

In [ ]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
def post_score(y_true, y_pred, threshold, metric_num):
  if metric_num == 1:
    return True if f1_score(y_true, y_pred) < threshold else False
  elif metric_num == 2:
    return True if accuracy_score(y_true, y_pred) < threshold else False
  elif metric_num == 3:
    return True if precision_score(y_true, y_pred) < threshold else False
  else:
    return True if recall_score(y_true, y_pred) < threshold else False

In [ ]:
## test case
y_true = [0, 1, 1, 0, 1, 1]
y_pred = [0, 1, 1, 0, 0, 1]
threshold = 0.7151669958566613
post_score(y_true, y_pred, threshold, 1)

In [ ]:
new_f1 = f1_score(y_test, y_pred_test)*(1-0.1)
print(new_f1)

In [ ]:
post_score(y_post, y_pred_post, new_f1, 2)

This told us that the model on post data still performs well.

### Pairwise Correlation

In [ ]:
# !pip3 install seaborn

In [ ]:
import seaborn as sns
X_post = post_data[numeric_features]
corr=X_post.corr()
mask = np.triu(np.ones_like(corr, dtype=bool))
f, ax = plt.subplots(figsize=(11, 9))
cmap = sns.diverging_palette(230, 20, as_cmap=True)
sns.heatmap(corr, mask=mask, cmap=cmap, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [ ]:
corr = X_train.drop(['artist_latitude', 'artist_longitude'], axis=1).corr()
corr_post = X_post.drop(['artist_latitude', 'artist_longitude'], axis=1).corr()

In [ ]:
len(corr.to_numpy())

In [ ]:
from numpy.linalg import norm
similaritys = []
for i in range(14):
  A = corr.to_numpy()[i]
  B = corr_post.to_numpy()[i]
  cosine = np.dot(A, B)/ (norm(A)*norm(B))
  similaritys.append(cosine)

In [ ]:
# [s for s in similaritys if s < 0.99]

In [ ]:
def corr_cosine_similarity(corr1, corr2, threshold):
  """compute the cosine similarity between two correlation coefficient dataframe
  corr1, corr2: dataframe
  threshold: for detecting cosine similarity between two arrays
  return if any cosine similarity falls below 0.99
  """
  from numpy.linalg import norm
  A = corr.to_numpy()
  B = corr_post.to_numpy()
  for i in range(len(A)):
    a = A[i]
    b = B[i]
    cosine = np.dot(a, b)/ (norm(a)*norm(b))
    if cosine < threshold:
      return True
    return False

In [ ]:
corr_cosine_similarity(corr, corr_post, 0.99)

In [ ]:
# c = corr.mask(np.eye(len(corr), dtype=bool)).abs()
# s = c.unstack()
# so = s.sort_values(kind="quicksort", ascending=False)
# so[so>0.6]

In [ ]:
# corr = X_train.drop(['artist_latitude', 'artist_longitude'], axis=1).corr()
# m = (corr.mask(np.eye(len(corr), dtype=bool)).abs() > 0.6).any()

In [ ]:
# raw = corr.loc[m, m]
# raw

In [ ]:
# raw_post = corr_post.loc[m, m]
# raw_post

In [ ]:
# similarity = pairwise.cosine_similarity(raw, raw_post)
# similarity

## Covariate Shift: Shift in the independent variables. 
### 1. KS-Test（two sample）
H0：two samples come from the same distribution.

H1：two samples' distribution are not the same.

If the p-value is less than .05, we reject the null hypothesis. We have sufficient evidence to say that the two sample datasets do not come from the same distribution.

In [ ]:
X_prev = pre_data.drop(columns = label)
X_post = post_data.drop(columns = label)

In [ ]:
from scipy.stats import ks_2samp
#return True means the program need to retrain
#k: at least k number of features are not from the same distribution
#X_prev,X_latter: dataframe
def KS_test(X_prev, X_latter, k):
    p_value = {'reject':0, 'accept':0}
    for i in X_prev.columns:
        p = ks_2samp(np.array(X_prev[i]), np.array(X_latter[i])).pvalue
        if p < 0.05:
            p_value['reject'] +=1
        else:
            p_value['accept'] +=1
    if p_value['reject'] > k:
        return True
    else: 
        return False

In [ ]:
#EXP: need to be retained
KS_test(X_prev, X_post, 2)

## 2. PSI
Interpretation

PSI < 0.1: no significant population change

PSI < 0.2: moderate population change

PSI >= 0.2: significant population change


In [ ]:
#Reference: https://github.com/mwburke/population-stability-index/blob/master/psi.py

def calculate_psi(expected, actual, buckettype='bins', buckets=10, axis=0):
    '''Calculate the PSI (population stability index) across all variables
    Args:
       expected: numpy matrix of original values
       actual: numpy matrix of new values, same size as expected
       buckettype: type of strategy for creating buckets, bins splits into even splits, quantiles splits into quantile buckets
       buckets: number of quantiles to use in bucketing variables
       axis: axis by which variables are defined, 0 for vertical, 1 for horizontal
    Returns:
       psi_values: ndarray of psi values for each variable
    Author:
       Matthew Burke
       github.com/mwburke
       worksofchart.com
    '''

    def psi(expected_array, actual_array, buckets):
        '''Calculate the PSI for a single variable
        Args:
           expected_array: numpy array of original values
           actual_array: numpy array of new values, same size as expected
           buckets: number of percentile ranges to bucket the values into
        Returns:
           psi_value: calculated PSI value
        '''

        def scale_range (input, min, max):
            input += -(np.min(input))
            input /= np.max(input) / (max - min)
            input += min
            return input


        breakpoints = np.arange(0, buckets + 1) / (buckets) * 100

        if buckettype == 'bins':
            breakpoints = scale_range(breakpoints, np.min(expected_array), np.max(expected_array))
        elif buckettype == 'quantiles':
            breakpoints = np.stack([np.percentile(expected_array, b) for b in breakpoints])


        expected_percents = np.histogram(expected_array, breakpoints)[0] / len(expected_array)
        actual_percents = np.histogram(actual_array, breakpoints)[0] / len(actual_array)

        def sub_psi(e_perc, a_perc):
            '''Calculate the actual PSI value from comparing the values.
               Update the actual value to a very small number if equal to zero
            '''
            if a_perc == 0:
                a_perc = 0.0001
            if e_perc == 0:
                e_perc = 0.0001

            value = (e_perc - a_perc) * np.log(e_perc / a_perc)
            return(value)

        psi_value = np.sum(sub_psi(expected_percents[i], actual_percents[i]) for i in range(0, len(expected_percents)))

        return(psi_value)

    if len(expected.shape) == 1:
        psi_values = np.empty(len(expected.shape))
    else:
        psi_values = np.empty(expected.shape[axis])
        print(psi_values)
    for i in range(0, len(psi_values)):
        if len(psi_values) == 1:
            psi_values = psi(expected, actual, buckets)
        elif axis == 0:
            psi_values[i] = psi(expected[:,i], actual[:,i], buckets)
        elif axis == 1:
            psi_values[i] = psi(expected[i,:], actual[i,:], buckets)

    return(psi_values)

In [ ]:
#return True means the program need to retrain
#k: at least k number of features has shift
def PSI_covariate(X_prev, X_latter, k):
    value = {'reject':0, 'accept':0}
    for i in X_prev.columns:
        v = calculate_psi(np.array(X_prev[i]), np.array(X_latter[i]))
        if v < 0.1:
            value['reject'] +=1
        else:
            value['accept'] +=1
    if value['reject'] > k:
        return True
    else: 
        return False

In [ ]:
#EXP: need to be retained
PSI_covariate(X_prev, X_post, 2)

## 3. Z-Score


In [ ]:
from scipy.stats import zscore

def Z_score_covariate(X_prev, X_post, k):
    # calculate Z-scores of the source and target datasets
    z_pre = zscore(X_prev)
    z_post = zscore(X_post)
    
    p_value = {'reject':0, 'accept':0}
    for i in range(z_pre.shape[1]):
        # perform a Kolmogorov-Smirnov test on the Z-score distributions
        p = ks_2samp(np.array(z_pre[:,i]), np.array(z_post[:,i])).pvalue
        if p < 0.05:
            p_value['reject'] +=1
        else:
            p_value['accept'] +=1
    if p_value['reject'] > k:
        return True
    else: 
        return False

In [ ]:
#EXP: need to be retained
# Z_score_covariate(X_prev, X_post, 2)

## Prior Probability Shift: shift of the target variable
### 1. KS-Test（two sample）
H0：two samples come from the same distribution.

H1：two samples' distribution are not the same.

If the p-value is less than .05, we reject the null hypothesis. We have sufficient evidence to say that the two sample datasets do not come from the same distributio

In [ ]:
y_prev = pre_data['popularity']
y_post = post_data['popularity']

In [ ]:
from scipy.stats import ks_2samp
#return True means the program need to retrain, check on the distribution of y
def KS_test_prior(y_prev, y_post):
    p = ks_2samp(np.array(y_prev), np.array(y_post)).pvalue
    if p < 0.05:
        return False
    else:
        return True

In [ ]:
KS_test_prior(y_prev, y_post)

### 2. PSI

In [ ]:
#return True means the program need to retrain, check on the distribution of y
def PSI_prior(y_prev, y_post):
    value = calculate_psi(np.array(y_prev), np.array(y_post))
    #no significant population change
    if value < 0.1:
        return False
    else:
        return True

In [ ]:
#EXP: no need to be retained
PSI_prior(y_prev, y_post)

### 3. Z-score

In [ ]:
from scipy.stats import zscore

def Z_score_prior(y_prev, y_post):
    # calculate Z-scores of the source and target datasets
    z_pre = zscore(y_prev)
    z_post = zscore(y_post)
    
    p = ks_2samp(np.array(z_pre), np.array(z_post)).pvalue

    if p < 0.05:
        return False
    else: 
        return True

In [ ]:
#EXP: no need to be retained
Z_score_prior(y_prev, y_post)

## Retraining Process


In [ ]:
merged_df = pre_data.append(post_data, ignore_index=True)
merged_df.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(merged_df.drop(columns = ['popularity']), merged_df['popularity'], test_size=0.3, random_state=42)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train_norm = scaler.fit_transform(X_train)
X_test_norm = scaler.transform(X_test)

In [ ]:
## random forest classfier
rf = RandomForestClassifier(random_state=42)
# Fit the model to the training data
rf.fit(X_train, y_train)
# Predict the labels of the test data
y_pred_rf = rf.predict(X_test)

In [ ]:
# !pip3 install xgboost

In [ ]:
## Xgboost
from xgboost import XGBClassifier
xgb = XGBClassifier(random_state=42)
xgb.fit(X_train, y_train) 
y_pred_xgb = rf.predict(X_test)

In [ ]:
## Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier(random_state=42).fit(X_train, y_train)
y_pred_gbc = gbc.predict(X_test)
# gbc.score(X_test, y_test)


In [ ]:
## KNN
# Use the KNN classifier to fit data:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

knn = KNeighborsClassifier()
knn.fit(X_train_norm, y_train) 

# Predict y data with classifier: 
y_predict_knn = knn.predict(X_test_norm)

In [ ]:
# cross validation - use acc 
from sklearn.metrics import make_scorer
metric = accuracy_score
rf_scores = cross_val_score(rf, X_test, y_test, cv=5, scoring=make_scorer(metric))
xgb_scores = cross_val_score(xgb, X_test, y_test, cv=5, scoring=make_scorer(metric))
gbc_scores = cross_val_score(gbc, X_test, y_test, cv=5, scoring=make_scorer(metric))
knn_scores = cross_val_score(knn, X_test_norm, y_test, cv=5, scoring=make_scorer(metric))


print("Random Fores mean score:", rf_scores.mean())
print("XGBoost mean score:", xgb_scores.mean())
print("Gradient Boosting mean score:", gbc_scores.mean())
print("KNN mean score:", knn_scores.mean())

In [ ]:
benchmark

In [ ]:
scores = [rf_scores.mean(), xgb_scores.mean(), gbc_scores.mean(), knn_scores.mean()]
models = [rf, xgb, gbc, knn]

best_score = max(scores)
best_model = models[scores.index(best_score)]

In [ ]:
print("Best Mean Score")
print(best_score)

In [ ]:
a = print(best_model)
a

In [ ]:
## use the best model and fine tuning the parameters
from sklearn.model_selection import RandomizedSearchCV
#the number of trees in the forest,
# the maximum depth of each tree, 
# the minimum number of samples required to split a node, 
# the minimum number of samples required to be at a leaf node, 
# and the number of features
param_space = {'n_estimators': [50, 100, 200, 300],
              'max_depth': [None, 5, 10, 20],
              'min_samples_split': [2, 5, 10, 15],
              'min_samples_leaf': [1, 2, 4, 8],
              'max_features': ['sqrt', 'log2', None]}

random_search = RandomizedSearchCV(
    estimator = best_model,
    param_distributions = param_space,
    n_jobs = 4,
    cv=5
)

random_search.fit(X_train, y_train)
best_params = random_search.best_params_
best_params

/usr/local/lib/python3.9/dist-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


In [ ]:
joblib.dump(random_search, "./retrained_model.joblib")

In [ ]:
import pickle
pickle.dump(, open(filename, 'wb'))

In [ ]:
# best_params

In [ ]:
# # Define the hyperparameter search space
# param_dist = {'n_estimators': [50, 100, 200, 300],
#               'max_depth': [None, 5, 10, 20],
#               'min_samples_split': [2, 5, 10, 15],
#               'min_samples_leaf': [1, 2, 4, 8],
#               'max_features': ['sqrt', 'log2', None]}

# # Initialize a RandomForestClassifier
# rfc = RandomForestClassifier(random_state=42)

# # Initialize a RandomizedSearchCV object with 10 iterations
# random_search = RandomizedSearchCV(rfc, param_distributions=param_dist, n_iter=10, cv=5)

# # Fit the RandomizedSearchCV object to the data
# random_search.fit(X_train, y_train)

# # Print the best hyperparameters found
# print(random_search.best_params_)

In [ ]:
# # Instantiate a Random Forest Classifier with 100 estimators
# rf_new = RandomForestClassifier(random_state=42)

# # Fit the model to the training data
# rf_new.fit(X_train, y_train)

# # Predict the labels of the test data
# y_pred = rf_new.predict(X_test)

# # Compute accuracy score
# accuracy = accuracy_score(y_test, y_pred)
# print("Random Forest Classifier Accuracy:", accuracy)

In [ ]:
print (f'Train Accuracy: {random_search.score(X_train,y_train):.3f}')
print (f'Test Accuracy: {random_search.score(X_test,y_test):.3f}')

In [ ]:
benchmark

In [ ]:
#previous
# print (f'Train Accuracy: {random_search.score(X_train,y_train):.3f}')
# print (f'Test Accuracy: {random_search.score(X_test,y_test):.3f}')

In [ ]:
Model_directory = tempfile.gettempdir()

In [ ]:
Model_directory

In [ ]:
version = 1


In [ ]:
export_path = os.path.join(Model_directory,str(version))
print('export_path = {}\n'.format(export_path))

In [ ]:
##import tensorflow as tf
#model = joblib.load("./deployed_model.joblib")
#tf.saved_model.save(model, '/content/drive/MyDrive/Capstone TCS/final/saved_model')

In [ ]:
rf.save(export_path)

